In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.datasets import mnist
import seaborn as sns
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
import plotly.graph_objects as go
import plotly.express as px
from tqdm import tqdm

In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
x_train = np.expand_dims(x_train, -1) # add channel dimension
x_test = np.expand_dims(x_test, -1)

In [3]:
latent_dim = 3

# latent_dim = 8  # Increased latent dimensions

# Encoder
def build_encoder():
    encoder_input = layers.Input(shape=(28, 28, 1))
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(1e-4))(encoder_input)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 2), padding='same')(x)
    x = layers.Dropout(0.3)(x)

    x = layers.Conv2D(128, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(1e-4))(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 2), padding='same')(x)
    x = layers.Dropout(0.3)(x)

    x = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(x)
    x = layers.Flatten()(x)
    latent_output = layers.Dense(latent_dim, name="latent_space")(x)
    return models.Model(encoder_input, latent_output, name="Encoder")

# Decoder
def build_decoder():
    decoder_input = layers.Input(shape=(latent_dim,))
    x = layers.Dense(7*7*256, activation='relu')(decoder_input)
    x = layers.Reshape((7, 7, 256))(x)
    
    x = layers.Conv2DTranspose(128, (3, 3), activation='relu', strides=2, padding='same')(x)
    x = layers.BatchNormalization()(x)
    
    x = layers.Conv2DTranspose(64, (3, 3), activation='relu', strides=2, padding='same')(x)
    x = layers.BatchNormalization()(x)

    decoder_output = layers.Conv2DTranspose(1, (3, 3), activation='sigmoid', padding='same')(x)
    return models.Model(decoder_input, decoder_output, name="Decoder")

# Full Autoencoder
encoder = build_encoder()
decoder = build_decoder()
autoencoder_input = encoder.input
autoencoder_output = decoder(encoder(autoencoder_input))
autoencoder = models.Model(autoencoder_input, autoencoder_output, name="Autoencoder")

autoencoder.compile(optimizer='adam', loss='mse')
autoencoder.summary()

callbacks = [
    ReduceLROnPlateau(monitor='loss', factor=0.25, patience=3, min_lr=1e-5, verbose=1),
    EarlyStopping(monitor='loss', patience=10, restore_best_weights=True, verbose=1)
]


Model: "Autoencoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 Encoder (Functional)        (None, 3)                 408067    
                                                                 
 Decoder (Functional)        (None, 28, 28, 1)         420353    
                                                                 
Total params: 828,420
Trainable params: 827,652
Non-trainable params: 768
_________________________________________________________________


In [4]:
autoencoder.fit(x_train, x_train, epochs=400, batch_size=128, callbacks=callbacks)


Epoch 1/400
469/469 [==============================] - 17s 20ms/step - loss: 0.0559 - lr: 0.0010
Epoch 2/400
469/469 [==============================] - 9s 20ms/step - loss: 0.0375 - lr: 0.0010
Epoch 3/400
469/469 [==============================] - 9s 19ms/step - loss: 0.0358 - lr: 0.0010
Epoch 4/400
469/469 [==============================] - 9s 19ms/step - loss: 0.0349 - lr: 0.0010
Epoch 5/400
469/469 [==============================] - 9s 19ms/step - loss: 0.0343 - lr: 0.0010
Epoch 6/400
469/469 [==============================] - 9s 19ms/step - loss: 0.0338 - lr: 0.0010
Epoch 7/400
469/469 [==============================] - 9s 19ms/step - loss: 0.0334 - lr: 0.0010
Epoch 8/400
469/469 [==============================] - 9s 19ms/step - loss: 0.0330 - lr: 0.0010
Epoch 9/400
469/469 [==============================] - 9s 19ms/step - loss: 0.0329 - lr: 0.0010
Epoch 10/400
469/469 [==============================] - 9s 19ms/step - loss: 0.0326 - lr: 0.0010
Epoch 11/400
469/469 [================

In [ ]:
autoencoder.save('autoencoder_model.h5')
encoder.save('encoder_model.h5')
decoder.save('decoder_model.h5')